In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
import torch
from torch.utils import data
from torch import nn
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import LinearConstraint, NonlinearConstraint, Bounds, minimize

# Load Data

In [13]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [14]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [15]:
config = initializeConfig()
obs = getObs(400)

In [16]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df.iloc[284]

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30584/2135078581.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[i].append(obs[i].mean()['Stability Margin (cal)'])


0         3.734223
1         7.875106
2         2.509849
3         7.372139
4         8.753891
5         7.977354
6         7.539283
7         8.558976
8     32187.640000
9         5.071216
10       99.001530
Name: 284, dtype: float64

In [17]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [21]:
filtered_df = df[df['Time'] > 90]
filtered_df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


# NN Stability Classifier

In [18]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [22]:
def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [33]:
classif = Classifier()
classif.double()

poly = PolynomialFeatures(2)

raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10
idxs = np.array(range(400)) #np.array(list(range(300, 400)) + list(range(200))) # FOR CV

train_X = torch.from_numpy(poly.fit_transform(raw_X.iloc[idxs])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y.iloc[idxs]])).double()

#test_X = torch.from_numpy(poly.fit_transform(raw_X[200:300])).double()
#test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[200:300]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [34]:
loss = nn.BCELoss()
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
def train(num_epochs, trainer, loss):
    for epoch in range(num_epochs):

        for X, y in data_iter:
            X.requires_grad = True
            l = loss(classif(X) ,y)

            trainer.zero_grad() #sets gradients to zero

            l.backward() # back propagation

            trainer.step() # parameter update

        l = loss(classif(train_X), train_Y)
        if epoch % 50 == 0:
            print(f'epoch {epoch + 1}, loss {l:f}')
            #print(X.grad)

In [35]:
train(num_epochs, trainer, loss), sum((classif(train_X) >= 0.5) == train_Y)/len(train_Y)

epoch 1, loss 0.363662
epoch 51, loss 0.039084
epoch 101, loss 0.018869
epoch 151, loss 0.001382
epoch 201, loss 0.000471


(None, tensor([1.]))

# Altitude Regression

In [36]:
poly = PolynomialFeatures(2)
raw = filtered_df.iloc[:,:8]
#qr = LinearRegression()
idxs = np.array(list(range(197)))# + list(range(50)))

ridge = Ridge(alpha=0.6)
ridge.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
#lasso = Lasso(alpha=.2)
#lasso.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])

#qr.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
print('R2', ridge.score(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3]))
#print('Test RMSE Quad', np.sqrt(MSE(filtered_df.iloc[150:, -3], qr.predict(poly.fit_transform(raw[150:])))))
#print('Test RMSE ridge', np.sqrt(MSE(filtered_df.iloc[150:, -3], ridge.predict(poly.fit_transform(raw[150:])))))
#print('Test RMSE lasso', np.sqrt(MSE(filtered_df.iloc[150:, -3], lasso.predict(poly.fit_transform(raw[150:])))))

print('Train RMSE', np.sqrt(MSE(filtered_df.iloc[idxs, -3], ridge.predict(poly.fit_transform(raw.iloc[idxs])))))

R2 0.9933903267684263
Train RMSE 1222.160451122849


# Solving COP

In [86]:
def stab_constraint(X):
    with torch.no_grad():
        return classif(torch.from_numpy(poly.fit_transform([X])))[0] - 0.6
def l0(X):
    return X[0]-2
def l1(X):
    return X[1]-2
def l2(X):
    return X[2]-2
def l3(X):
    return X[3]-2
def l4(X):
    return X[4]-2
def l5(X):
    return X[5]-2
def l6(X):
    return X[6]-2
def l7(X):
    return X[7]-2

def u0(X):
    return 10-X[0]
def u1(X):
    return 10-X[1]
def u2(X):
    return 10-X[2]
def u3(X):
    return 10-X[3]
def u4(X):
    return 10-X[4]
def u5(X):
    return 10-X[5]
def u6(X):
    return 10-X[6]
def u7(X):
    return 10-X[7]
def objective(X):
    return -ridge.predict(poly.fit_transform([X]))[0]
classif.eval()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [111]:
ineq_cons = {'type': 'ineq',
             'fun' : stab_constraint}
L0 = {'type': 'ineq',
             'fun' : l0}
L1 = {'type': 'ineq',
             'fun' : l1}
L2 = {'type': 'ineq',
             'fun' : l2}
L3 = {'type': 'ineq',
             'fun' : l3}
L4 = {'type': 'ineq',
             'fun' : l4}
L5 = {'type': 'ineq',
             'fun' : l5}
L6 = {'type': 'ineq',
             'fun' : l6}
L7 = {'type': 'ineq',
             'fun' : l7}
U0 = {'type': 'ineq',
             'fun' : u0}
U1 = {'type': 'ineq',
             'fun' : u1}
U2 = {'type': 'ineq',
             'fun' : u2}
U3 = {'type': 'ineq',
             'fun' : u3}
U4 = {'type': 'ineq',
             'fun' : u4}
U5 = {'type': 'ineq',
             'fun' : u5}
U6 = {'type': 'ineq',
             'fun' : u6}
U7 = {'type': 'ineq',
             'fun' : u7}
# x = minimize(objective, X0, method='trust-constr', options = {'disp':True}).x#, constraints = [ineq_cons], options={'ftol': 1e-9, 'disp': True})

In [120]:
import warnings
warnings.filterwarnings("ignore")

samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
results = []
for i in range(len(samples)):
    X0 = np.array(samples.iloc[i])
    res = minimize(objective, X0, method='COBYLA', 
               constraints = [ineq_cons, L0, L1, L2, L3, L4, L5, L6, L7, U0, U1, U2, U3, U4, U5, U6, U7], 
               options = {'verbose':1, 'display':1, 'maxiter':10}) # constraints = [constraint], options = {'verbose':1})#, bounds = bounds)
    results.append((X0, res.x, -res.fun))

In [121]:
sorted(results, key = lambda x: x[2], reverse = True)[:10]

[(array([3.84330328, 3.11903088, 8.63367165, 8.88974916, 8.81972552,
         3.69938965, 8.75334237, 8.74292895]),
  array([3.76679581, 2.15874082, 9.83011208, 8.85537824, 8.78194412,
         3.52422344, 9.76373191, 9.74383055]),
  100647.45247722218),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.87590522, 2.        , 7.79436646, 7.1287923 , 5.34349866,
         3.48989421, 5.66564727, 3.70702818]),
  99108.85424540826),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.87590522, 2.        , 7.79436646, 7.1287923 , 5.34349866,
         3.48989421, 5.66564727, 3.70702818]),
  99108.85424540826),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.87590522, 2.        , 7.79436646, 7.1287923 , 5.34349866,
         3.48989421, 5.66564727, 3.70702818]),
  991

array([7.10731359, 4.39252568, 6.82182993, 3.48153784, 3.91557955,
       5.82419792, 2.60837853, 4.97694776])

In [78]:

classif(torch.from_numpy(poly.fit_transform([X0])))[0]

tensor([1.0000], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [79]:
torch.from_numpy(poly.fit_transform([X0]))

tensor([[ 1.0000,  2.2687,  5.6727,  9.0363,  2.6363,  9.0568,  5.0017,  2.8575,
          8.3474,  5.1469, 12.8696, 20.5006,  5.9809, 20.5470, 11.3474,  6.4829,
         18.9376, 32.1797, 51.2605, 14.9549, 51.3766, 28.3734, 16.2101, 47.3524,
         81.6554, 23.8224, 81.8402, 45.1974, 25.8218, 75.4299,  6.9500, 23.8763,
         13.1860,  7.5333, 22.0062, 82.0254, 45.2997, 25.8802, 75.6006, 25.0174,
         14.2927, 41.7515,  8.1656, 23.8531, 69.6791]], dtype=torch.float64)

In [106]:
df.max()

Schord             9.993589
Sspan              9.946829
Ssweep             9.977092
Stip               9.993466
Bchord             9.989678
Bspan              9.979451
Bsweep             9.988933
Btip               9.999828
Schordspan        97.753953
Schordsweep       96.163414
Schordtip         97.665309
Sspansweep        97.321714
SSpantip          98.412780
SSweeptip         98.140227
Bchordspan        97.043274
Bchordsweep       99.053989
Bchordtip         98.708702
Bspansweep        96.466173
BSpantip          97.026381
BSweeptip         95.959608
Altitude       87504.620000
Stability          9.438525
Time             164.991800
dtype: float64